**Задание 1.**

Цель: Использование метода опорных векторов (SVM) для классификации видов ирисов на основе их характеристик.



Данные: Набор данных "Ирисы", состоящий из 150 наблюдений, каждое из которых содержит четыре атрибута (длина и ширина чашелистиков, длина и ширина лепестков) и метку класса, указывающую на принадлежность к одному из трех видов ирисов (Iris setosa, Iris virginica, Iris versicolor).



Задачи:

Разделите данные на признаки (X) и целевую переменную (y), где признаки - это атрибуты ирисов, а целевая переменная - метка класса
Далее разделите полученные данные на обучающую и тестовую выборки, используя соотношение 70% на 30% соответственно. Установите random_state равным 105 для воспроизводимости результатов.
Масштабируйте признаки, используя MinMaxScaler, чтобы привести все значения к диапазону от 0 до 1. Это необходимо для улучшения производительности модели SVM.
Обучите модель SVM с радиально-базисной функцией (RBF) и коэффициентом регуляризации C, равным 1, на обучающей выборке.
После обучения модели выполните предсказание на тестовой выборке и оцените качество модели с помощью classification_report и confusion_matrix. Анализируйте результаты, сосредоточив внимание на точности, полноте и F1-мере для каждого из классов, а также на общей точности модели.

Дополнительные указания:

Убедитесь, что перед выполнением масштабирования вы правильно разделили данные на обучающую и тестовую выборки, чтобы предотвратить утечку данных.

Использование радиально-базисной функции (RBF) в качестве ядра SVM предполагает, что вы рассматриваете возможность нелинейной разделимости данных. Обратите внимание на результаты и рассмотрите возможность экспериментирования с другими типами ядер и параметрами C для улучшения результатов классификации.

In [41]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [42]:
data = datasets.load_iris()

features = data['data']
cols = data['feature_names']
target = data['target']
target_name = data['target_names']

dataset = pd.DataFrame(features, columns= cols)
dataset['target'] = target

target_dict = {i: target_name[i] for i in range(len(target_name))}
dataset['target_name'] = dataset['target'].map(target_dict)

X = dataset.drop(columns=['target', 'target_name'])
y = dataset[['target', 'target_name']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=105)

scaler = MinMaxScaler()
X_tr_scaled = scaler.fit_transform(X_train)
X_t_scaled = scaler.transform(X_test)

model = SVC(kernel='rbf', C=1)
model.fit(X_tr_scaled, y_train['target'])
prediction = model.predict(X_t_scaled)

print(f"Report:\n{classification_report(y_test['target'], prediction)}")
print(f"Confusion matrix:\n{confusion_matrix(y_test['target'], prediction)}")

Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      0.94      0.97        16
           2       0.94      1.00      0.97        17

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

Confusion matrix:
[[12  0  0]
 [ 0 15  1]
 [ 0  0 17]]


In [43]:
model = SVC()
params = {'degree': range(3, 9),
          "C": [0.1, 0.01, 1, 2],
         'kernel': ['rbf', 'poly', 'linear']}
model_search = GridSearchCV(param_grid=params, estimator=model, scoring='neg_mean_squared_error', cv=3)
model_search.fit(X_tr_scaled, y_train['target'])

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 0.01, 1, 2], 'degree': range(3, 9),
                         'kernel': ['rbf', 'poly', 'linear']},
             scoring='neg_mean_squared_error')

In [44]:
model_search.best_params_

{'C': 2, 'degree': 3, 'kernel': 'linear'}

In [45]:
model = SVC(kernel='linear', C=2, degree=3)
model.fit(X_tr_scaled, y_train['target'])
prediction = model.predict(X_t_scaled)

print(f"Report:\n{classification_report(y_test['target'], prediction)}")
print(f"Confusion matrix:\n{confusion_matrix(y_test['target'], prediction)}")

Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      0.94      0.97        16
           2       0.94      1.00      0.97        17

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

Confusion matrix:
[[12  0  0]
 [ 0 15  1]
 [ 0  0 17]]


**Задача 2:**

Цель задачи: Использование метода опорных векторов для регрессии (SVR) для предсказания медианных цен на жилье в Калифорнии на основе социально-экономических показателей различных блоков населения.

Задачи:

Разделите данные на обучающую и тестовую выборки в соотношении 70% на 30% со значением random_state=105 для воспроизводимости результатов.
Произведите масштабирование признаков с использованием MinMaxScaler для улучшения производительности модели SVR. Убедитесь, что масштабирование выполняется отдельно для обучающей и тестовой выборок, чтобы избежать утечки данных.
Обучите модель SVR с полиномиальным ядром (kernel='poly'), начальными параметрами степени ядра degree=3 и коэффициентом регуляризации C=1, на обучающей выборке.
Выполните предсказание медианных цен на жилье для тестовой выборки и оцените качество модели, вычислив корень из среднеквадратичной ошибки (RMSE).
Используйте GridSearchCV для автоматического подбора оптимальных параметров модели SVR, варьируя degree и C. Определите лучшую комбинацию параметров на основе кросс-валидации.

In [46]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error as MSE, r2_score

data = datasets.fetch_california_housing()

features = data['data']
cols = data['feature_names']
target = data['target']

dataset = pd.DataFrame(features, columns= cols)
dataset['target'] = target

X = dataset.drop(columns=['target'])
y = dataset[['target']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=105)

scaler = MinMaxScaler()
X_tr_scaled = scaler.fit_transform(X_train)
X_t_scaled = scaler.transform(X_test)

model = SVR(kernel='poly', C=1, degree=3)
model.fit(X_tr_scaled, y_train['target'])
prediction = model.predict(X_t_scaled)

print(f"Качество:\n{r2_score(y_test, prediction)}")
print(f"RMSE:\n{MSE(y_test, prediction) ** (1/2)}")

Качество:
0.5963673441239626
RMSE:
0.7366135199244025


In [47]:
model = SVR(kernel='linear')
params = {'degree': range(3, 9),
          "C": [0.1, 0.01, 1, 2, 3, 0.5, 0.75],}
model_search = GridSearchCV(param_grid=params, estimator=model, scoring='neg_mean_squared_error', cv=3)
model_search.fit(X_tr_scaled, y_train['target'])

GridSearchCV(cv=3, estimator=SVR(kernel='linear'),
             param_grid={'C': [0.1, 0.01, 1, 2, 3, 0.5, 0.75],
                         'degree': range(3, 9)},
             scoring='neg_mean_squared_error')

In [48]:
model_search.best_params_

{'C': 3, 'degree': 3}

In [49]:
model = SVR(kernel='poly', C=3, degree=3)
model.fit(X_tr_scaled, y_train['target'])
prediction = model.predict(X_t_scaled)

print(f"Качество:\n{r2_score(y_test, prediction)}")
print(f"RMSE:\n{MSE(y_test, prediction) ** (1/2)}")

Качество:
0.5987569901261652
RMSE:
0.7344297786127988
